In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
#os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"

In [3]:
#changing the working directory
#%cd /content/gdrive/My Drive/kaggle

In [4]:
#!kaggle datasets download -d mlg-ulb/creditcardfraud

In [5]:
#unzipping the zip files and deleting the zip files
#!unzip \*.zip  && rm *.zip

In [6]:
!pwd

/content


In [32]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings('ignore')

In [33]:
credit = pd.read_csv('/content/gdrive/My Drive/kaggle/creditcard.csv')
credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [34]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [35]:
credit.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [36]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [37]:
# 27 most important features according to our EDA
cols = ['V'+str(i) for i in range(1, 29) if i != 25]
print(cols)

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V26', 'V27', 'V28']


In [38]:
# selecting the 19 most important features according to our EDA 
X = credit[cols]
print('x', X.shape)
y = credit['Class'] # selecting the target variable
print('y', y.shape)

x (284807, 27)
y (284807,)


In [39]:
# computing the class weight

val_count = credit['Class'].value_counts()
weights = dict(1 / val_count)
weights

{0: 3.51722561243691e-06, 1: 0.0020325203252032522}

In [40]:
# adminstrative cost
admin_cost = 2.5

In [41]:
# defining a function to calculate cost savings
def cost_saving(ytrue, ypred, amount):
    fp = np.sum((ytrue == 0) & (ypred == 1))
    cost = np.sum(fp*admin_cost) + np.sum((amount[(ytrue == 1) & (ypred == 0)]))
    max_cost = np.sum((amount[(ytrue == 1)]))
    savings = 1 - (cost/max_cost)
    
    return savings

In [42]:
cv = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)

In [100]:
# defining a function to calculate cost saving per fold (splits) of our cv
def cost_saving_per_split(scores, x, y, cv_object):
    results = []
    for i, (_, test_ind) in zip(range(cv_object.n_splits), cv_object.split(x, y)):
        ypred = scores['estimator'][i].predict(x[test_ind])
        ytrue = y[test_ind]
        amount = credit['Amount'].values[test_ind]
        results.append(cost_saving(ytrue, ypred, amount))
        
    return results

In [99]:
# defining a function to return a dataframe of metrics results for each fold in our cv
def get_metric_scores(scores, x, y=y, cv_object=cv):
    ind = ['split_'+str(n) for n in range(1, cv_object.n_splits+1)]
    
    scores_credit = pd.DataFrame(index=ind)
    
    scores_credit['f1_score'] = scores['test_f1']
    scores_credit['auc_pr'] = scores['test_average_precision']
    scores_credit['cost_savings'] = cost_saving_per_split(scores, x, y, cv_object)

    return scores_credit

**Fraud Sensitive model (Not cost sensitive)**

In [46]:
fraud_sensitive_model = GradientBoostingClassifier()
fraud_sensitive_scaler = StandardScaler()

fraud_sensitive_pipe = Pipeline([('scaler', fraud_sensitive_scaler), ('model', fraud_sensitive_model)])

fraud_sensitive_scores = cross_validate(fraud_sensitive_pipe, np.array(X), y, 
                                        scoring=['f1', 'average_precision'], cv=cv, n_jobs=4, 
                                        return_estimator=True, error_score='raise')

In [50]:
fraud_sensitive_results = get_metric_scores(fraud_sensitive_scores, np.array(X))
fraud_sensitive_results

,f1_score,auc_pr,cost_savings
split_1,0.694836,0.654883,0.579972
split_2,0.779221,0.624357,0.736572
split_3,0.712871,0.566435,0.547409
split_4,0.406977,0.208251,0.381757


In [51]:
fraud_sensitive_results.mean()

f1_score        0.648476
auc_pr          0.513481
cost_savings    0.561427
dtype: float64

**Cost Sensitive model**

In [105]:
#we will use sample weight to penalize our model accordingly. admin cost for False positive and transactional cost (amount) for False Neg.

sample_weights = np.array([credit['Amount'][ind] if fraud else admin_cost for ind, fraud in enumerate(y)])

In [106]:
scaler2 = StandardScaler()
cost_sensitive_model = GradientBoostingClassifier()

cost_sensitive_pipe = Pipeline([('scaler', scaler2), ('model', cost_sensitive_model)])

cost_sensitive_scores = cross_validate(cost_sensitive_pipe, np.array(X), y, 
                                       scoring=['f1', 'average_precision'], cv=cv, n_jobs=4, return_estimator=True, 
                                       fit_params={'model__sample_weight': sample_weights}, error_score='raise')

In [107]:
cost_sensitive_results = get_metric_scores(cost_sensitive_scores, np.array(X))
cost_sensitive_results

,f1_score,auc_pr,cost_savings
split_1,0.764940,0.684253,0.736267
split_2,0.776860,0.691438,0.659827
split_3,0.809917,0.786029,0.801796
split_4,0.791667,0.763994,0.712838


In [108]:
cost_sensitive_results.mean()

f1_score        0.785846
auc_pr          0.731429
cost_savings    0.727682
dtype: float64

**Cost Sensitive Model - SMOTE**

In [109]:
# this custom pipeline is needed to be able to access the coef_ attribute of the logistic regressor, tree based
# because RFECV doesn't allow pipelines that can't access the coef_ or feature_importances_ attributes
# of the last estimator (model)
class ImbModelPipeline(Pipeline):
    # incase the model is a logistic based model
    @property
    def coef_(self):
        return self.steps[-1][1].coef_
    
    # in case the model is a tree based model
    @property
    def feature_importances_(self):
        return self.steps[-1][1].feature_importances_

In [110]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=1)
smote_model = GradientBoostingClassifier()
scaler3 = StandardScaler()

pipe_smote = ImbModelPipeline([('scaler', scaler3), ('smote', smote), ('model', smote_model)])

smote_scores = cross_validate(pipe_smote, np.array(X), y, scoring=['f1', 'average_precision'], 
                              cv=cv, n_jobs=4, return_estimator=True, error_score='raise')


In [111]:
smote_results = get_metric_scores(smote_scores, np.array(X))
smote_results

,f1_score,auc_pr,cost_savings
split_1,0.199811,0.696239,0.662549
split_2,0.201802,0.791970,0.718173
split_3,0.178626,0.825165,0.753478
split_4,0.203422,0.722902,0.656561


In [112]:
smote_results.mean()

f1_score        0.195915
auc_pr          0.759069
cost_savings    0.697690
dtype: float64

**Cost sensitive - Undersampling**

In [113]:
from imblearn.under_sampling import RandomUnderSampler

undersampler = RandomUnderSampler()
undersample_model = GradientBoostingClassifier()
scaler4 = StandardScaler()

pipe_under = Pipeline([('undersampler', undersampler),('scaler', scaler4), ('model', undersample_model)])

undersample_scores = cross_validate(pipe_under, np.array(X), y, scoring=['f1', 'average_precision'], 
                                    cv=cv, n_jobs=4, return_estimator=True,)


In [114]:
undersample_results = get_metric_scores(undersample_scores, np.array(X))
undersample_results

,f1_score,auc_pr,cost_savings
split_1,0.075714,0.601562,0.416351
split_2,0.095358,0.714012,0.450285
split_3,0.078000,0.759344,0.441488
split_4,0.075510,0.539292,0.435264


In [115]:
undersample_results.mean()

f1_score        0.081146
auc_pr          0.653553
cost_savings    0.435847
dtype: float64